### Practice: Large Language Models and Their Implications

<!-- ![img](https://substackcdn.com/image/fetch/f_auto,q_auto:good,fl_progressive:steep/https%3A%2F%2Fbucketeer-e05bbc84-baa3-437e-9518-adb32be77984.s3.amazonaws.com%2Fpublic%2Fimages%2F4470ce74-e595-4750-92a5-5f21f040df6d_577x432.jpeg) -->
![img](https://i.imgur.com/QGYa2J8.jpeg)

In this notebook, you're gonna play with some of the largest language models on the Internet.

_Based on works of: Tim Dettmers, Artem Chumachenko, Younes Belkada, Felix Marty, Yulian Gilyazev, Gosha Zolotov, Andrey Ishutin, Elena Volf, Artemiy Vishnyakov, Svetlana Shirokovskih.

### Part 1: prompt engineering (3 points total)

In the assignment, we'll use public APIs that host the 100B+ models for inference. Your task is to prompt-engineer the model into solving a few tasks for you.


__Which API?__ You are free to use any publicly available API. Here's a few options:

- BLOOM API - [bigscience/bloom](https://huggingface.co/bigscience/bloom) (on the right; recommended)
- OpenAI API (via VPN) - [openai.com/api](https://openai.com/api/)
- AI21 Jurrasic API - [ai21.com](https://www.ai21.com/blog/announcing-ai21-studio-and-jurassic-1)

These APIs may require you to create a (free) account on their platform. Please note that some APIs also have paid subscriptions. __You do not need to pay them__, this assignment was designed to be solved using free-tier subscriptions. If no APIs work for you, you can also solve these tasks with the 6.7B model that you will find later in this notebook - but this will make the tasks somewhat harder.

__Quests:__ you will need to solve 4 problems. For each one, please attach a short __description__ of your solution and a __screenshot__ from the API you use. _[If you use python APIs, show your python code with outputs]_

__Example:__ Tony is talking to Darth Vader ([BLOOM API](https://huggingface.co/bigscience/bloom)). Black text is written manually, blue text is generated.
<hr>

![img](https://i.imgur.com/a1QhKF7.png)
<hr>

__It is fine to roll back a few times,__ e.g. in the example above, the model first generated Vader lines twice in a row, and we rolled that back. However, if you need more than 1-2 rollbacks per session, you should probably try a different prompt.

__Task 1 (? pt):__ arange a conversation between any two of the following:

- a celebrity or politician of your choice
- any fictional character (except Darth Vader)
- yourself

Compare two setups: a) you prompt with character names only b) you supply additional information (see example).

In [ ]:
# <your code OR writeup with screenshots>

__Please choose task 2a or 2b (1pt)__ depending on your model (you can do both, but you will be awarded points for one of these two tasks).

__Task 2a: (for BLOOM)__ zero-shot translation. Take the first verse of [Edgar Allan Poe's "Raven"](https://www.poetryfoundation.org/poems/48860/the-raven) and __translate it into French.__ (You are free to use any other text of at least the same size)

Original text: ```
Once upon a midnight dreary, while I pondered, weak and weary,
Over many a quaint and curious volume of forgotten lore—
    While I nodded, nearly napping, suddenly there came a tapping,
As of some one gently rapping, rapping at my chamber door.
“’Tis some visitor,” I muttered, “tapping at my chamber door—
            Only this and nothing more.”
```

Verify your translation by converting french back into english using a public machine translation service.

__Task 2b: (non-BLOOM):__ toxicity classification for [SetFit/toxic_conversations](https://huggingface.co/datasets/SetFit/toxic_conversations). Make the model solve binary classification (toxic vs not toxic) in the few shot mode. For few-shot examples, use 2-3 toxic and 2-3 non-toxic non-toxic examples. Measure accuracy on at least 25 samples. You may need to try several different prompts before you find the one that works.

In [ ]:
# <your code OR writeup with screenshots>


__Task 3 (1pt):__ create a prompt and few-shot examples tha make the model __change the gender pronouns__ of the main actor in a given sentence in any direction of your choice. E.g. the doctor took off _his_ mask <-> the doctor took of _her_ mask.


In [ ]:
# <your code OR writeup with screenshots>

__Task 4 (1pt):__ write a prompt and supply examples such that the model would __convert imperial units to metric units__ (miles -> kilometers; mph -> kph). More specifically, the model should rewrite a given sentence and replace all imperial units with their metric equivalents. After it works with basic distances and speed, try to find complicated examples where it does *not* work.

Please note that 1 mile is not equal to 1 km :)

In [ ]:
# <your code OR writeup with screenshots>

### Part 2: local inference

Now, let's try and load the strongest model that can fit a typical Colab GPU (T4 with 16 GB as of spring 2023). 

Our best candidates are the smaller versions of the best performing open source models: 
- 7 Bn parameters version of [LLaMA](https://arxiv.org/pdf/2302.13971.pdf) - best for spring 2023, released by Facebook
- 7 Bn parameters version of [Falcon](https://falconllm.tii.ae) - close competitor to Llama, released in May 2023 by [Technology Innovation Institute of UAE](https://www.tii.ae).
- 6.7 Bn parameters version of [OPT](https://arxiv.org/abs/2205.01068) - top choice in this nomination in 2022, released by Facebook.

Beware: while these models are smaller than the ones in API, they're still over 60x larger than the BERT we played with last time. The code below will *just barely* fit into memory, so make sure you don't have anything else loaded. Sometimes you may need to restart runtime for the code to work.

It's a good time to restart your kernel and switch to GPU! (Runtime -> Change runtime type)
<center><img src="https://i.imgur.com/OOfDYzJ.png" width=240px></center>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 

if torch.cuda.get_device_capability() < (7, 5):
    raise ValueError(f"You got a GPU with capability {torch.cuda.get_device_capability()}, need at least (7, 5)")
else: 
    print(f"Pytorch version {torch.__version__} imported OK. \ndevice = {device}")

# Note: this code requires a Turing GPU or newer. Good: T4, RTX 20xx/30xx, A100/Axx; Bad: K80, P100, V100
# Colab gives you T4. If you get older GPUs, please wait or switch to a new account (don't use both at the same time)

In [ ]:
%pip install --quiet bitsandbytes==0.39.0 transformers==4.29.2 datasets==2.12.0 accelerate==0.19.0 sentencepiece==0.1.99 einops==0.6.1
!pip list | grep -e "bitsandbytes" -e "transformers" -e datasets -e torch -e accelerate -e sentencepiece -e einops

In [ ]:
import transformers
from datasets import load_dataset
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, LlamaTokenizer, GenerationConfig
import bitsandbytes as bnb

from tqdm.auto import tqdm, trange

In [ ]:
model_name = 'decapoda-research/llama-7b-hf'  # published in 03-2023 model best in class among openly available

model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    load_in_8bit=True, 
    device_map='auto',
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True, 
    offload_state_dict=True,
    trust_remote_code=True
    )

# note: the flags `torch_dtype`, `low_cpu_mem_usage`, `offload_state_dict`, `load_in_8bit` slow down the code to save RAM; remove them if you have >32GB RAM

In [ ]:
#  loading Llama tokenizer...
tokenizer = LlamaTokenizer.from_pretrained(model_name, device_map=device)
tokenizer.pad_token_id = tokenizer.eos_token_id

In [ ]:
for module in model.modules():
    if isinstance(module, bnb.nn.Linear8bitLt):
        module.state.memory_efficient_backward = True

for param in model.parameters():
    param.requires_grad = False  # freeze the model - train adapters later
    if param.ndim == 1:
        # cast the small parameters (e.g. layernorm) to fp32 for stability
        param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations, comment out if you have >32GB RAM

In [ ]:
# code to save from RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

class AddInputGrad(nn.Sequential):
    def forward(self, x): 
        return super().forward(x).requires_grad_(True)
    
model.model.embed_tokens = AddInputGrad(model.model.embed_tokens)

## Text generation

**Comparison of strategies for language model text generation:**

| Strategy | Description | Pros & Cons |
| --- | --- | --- |
| Greedy Search | Chooses the word with the highest probability as the next word in the sequence. | **Pros:** Simple and fast. <br> **Cons:** Can lead to repetitive and incoherent text. |
| Sampling with Temperature | Introduces randomness in the word selection. A higher temperature leads to more randomness. | **Pros:** Allows exploration and diverse output. <br> **Cons:** Higher temperatures can lead to nonsensical outputs. |
| Nucleus Sampling (Top-p Sampling) | Selects the next word from a truncated vocabulary, the "nucleus" of words that have a cumulative probability exceeding a pre-specified threshold (p). | **Pros:** Balances diversity and quality. <br> **Cons:** Setting an optimal 'p' can be tricky. |
| Beam Search | Explores multiple hypotheses (sequences of words) at each step, and keeps the 'k' most likely, where 'k' is the beam width. | **Pros:** Produces more reliable results than greedy search. <br> **Cons:** Can lack diversity and lead to generic responses. |
| Top-k Sampling | Randomly selects the next word from the top 'k' words with the highest probabilities. | **Pros:** Introduces randomness, increasing output diversity. <br> **Cons:** Random selection can sometimes lead to less coherent outputs. |
| Length Normalization | Prevents the model from favoring shorter sequences by dividing the log probabilities by the sequence length raised to some power. | **Pros:** Makes longer and potentially more informative sequences more likely. <br> **Cons:** Tuning the normalization factor can be difficult. |
| Stochastic Beam Search | Introduces randomness into the selection process of the 'k' hypotheses in beam search. | **Pros:** Increases diversity in the generated text. <br> **Cons:** The trade-off between diversity and quality can be tricky to manage. |
| Decoding with Minimum Bayes Risk (MBR) | Chooses the hypothesis (out of many) that minimizes expected loss under a loss function. | **Pros:** Optimizes the output according to a specific loss function. <br> **Cons:** Computationally more complex and requires a good loss function. |

Documentation references:
- [reference for `AutoModelForCausalLM.generate()`](https://huggingface.co/docs/transformers/v4.29.1/en/main_classes/text_generation#transformers.GenerationMixin.generate)
- [reference for `AutoTokenizer.decode()`](https://huggingface.co/docs/transformers/main_classes/tokenizer#transformers.PreTrainedTokenizer.decode)
- Huggingface [docs on generation strategies](https://huggingface.co/docs/transformers/generation_strategies)

In [ ]:
prompt = 'The first discovered martian lifeform looks like '
max_new_tokens = 64
batch = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False).to(device)
batch

#### greedy generation

In [ ]:
%%time
with torch.cuda.amp.autocast():
    output_tokens = model.generate(**batch, max_new_tokens=max_new_tokens, do_sample=False)

print(tokenizer.decode(output_tokens[0].cpu()))

In [ ]:
# this is a detailed code for text generation

# prompt1 = "Moscow is the capital of"
prompt1 = "Skippy, a young android, likes to dream about electric"

print(prompt1, '\n')

voc = tokenizer.get_vocab()
voc_rev = {v:k for k, v in voc.items()}  # reverse vocab for decode
past_key_values = None 
n = 5
temperature = 1.0

for i in range(10):
    with torch.cuda.amp.autocast():
        batch1 = tokenizer(prompt1, return_tensors='pt', return_token_type_ids=False).to(device)
        outputs = model.forward(**batch1, use_cache=True,)
        logits = outputs.logits[0, -1, :] / temperature

        probs = torch.nn.functional.softmax(logits, dim=-1)  
        
        # GREEDY GENERATION
        next_token_id = logits.argmax(-1)
        
        # SAMPLING GENERATION
        # next_token_id = torch.multinomial(probs, num_samples=1)

        next_token = tokenizer.decode(next_token_id)
        prompt1 += next_token        
        
        # next_token = voc_rev[next_token_id.item()]  # uncomment to show raw tokens
        
        sorted_probs, sorted_indices = torch.sort(probs, descending=True)
        # cumulative_probs = torch.cumsum(sorted_probs, dim=-1)
        
        tokens0 = sorted_indices[:n]
        for t, p in zip (tokens0, probs1):
            t = voc_rev[t.item()]
            # t = tokenizer.decode([t])
            print(f"{t:<10}: {p:.4f} ")
            
        print('\t\t\t', f"{next_token}", end='\n', flush=True)

#### sampling generation

In [ ]:
with torch.cuda.amp.autocast():
    output_tokens = model.generate(**batch, max_new_tokens=max_new_tokens, do_sample=True)

print(tokenizer.decode(output_tokens[0].detach().cpu().numpy()))

#### sampling generation with temperature

In [ ]:
# moderate temperature makes generated text more diverse
with torch.cuda.amp.autocast():
    output_tokens = model.generate(**batch, max_new_tokens=max_new_tokens, do_sample=True, temperature = 2.0)

print(tokenizer.decode(output_tokens[0].detach().cpu().numpy()))

In [ ]:
# tool high a temperature makes the model generate unrelated text
with torch.cuda.amp.autocast():
    output_tokens = model.generate(**batch, max_new_tokens=max_new_tokens, do_sample=True, temperature = 8.0)

print(tokenizer.decode(output_tokens[0].detach().cpu().numpy()))

In [ ]:
# Low temperature brings results closer to greedy search
with torch.cuda.amp.autocast():
    output_tokens = model.generate(**batch, max_new_tokens=max_new_tokens, do_sample=True, temperature = 0.3)

print(tokenizer.decode(output_tokens[0].detach().cpu().numpy()))

#### beam search

In [ ]:
# beam search is best for high quality longer generation
with torch.cuda.amp.autocast():
    output_tokens = model.generate(**batch, max_new_tokens=max_new_tokens * 2, num_beams=4)

print(tokenizer.decode(output_tokens[0].detach().cpu().numpy()))

**Task 7 write code for nucleus sampling generation (2 points)**:

**Bonus task: write code for beam search (extra 2 points)**

In [ ]:
def nucleus_sampling(model, tokenizer, prompt, prob=0.5):
    """generates the next token from the nucleus of tokens with cumulative probability up to param:prob"""
    
    <YOUR CODE HERE>
        
    return sampled_token, possible_tokens

In [ ]:
# Tests for nucleus sampling
torch.manual_seed(42)

test_prompt = "Elbrus is the highest"
next_token, possible_tokens = nucleus_sampling(model, tokenizer, test_prompt, prob=0.9)
print(test_prompt, next_token, possible_tokens)
assert next_token == 'mountain'
assert set(possible_tokens) == {'mountain', 'peak', 'point'}
assert len(possible_tokens) == 3

test_prompt = "Large language models can learn to"
next_token, possible_tokens = nucleus_sampling(model, tokenizer, test_prompt, prob=0.3)
print(test_prompt, next_token, possible_tokens)
assert next_token == 'generate'
assert set(possible_tokens) == {'generate', 'perform', 'predict', 'understand'}
assert len(possible_tokens) == 4

### Part 3: parameter-efficient fine-tuning

__Task 8: Parameter-efficient finetuning with LoRA (1 point)__

Since the model barely fits into memory, we won't be able to train it with conventional fine-tuning. Instead, you can use low-rank adapters based on [LoRA paper](https://arxiv.org/pdf/2106.09685.pdf).

The core idea is to add low-rank adapters __in parallel with attention projection matrices,__ like this:
<center><img src="https://i.imgur.com/6bQLNiG.png" width=240px></center>

In [ ]:
class LoRALayer(nn.Module):
    """Wraps a linear layer with LoRA-like adapter. Wraps an existing OPT linear layer"""
    def __init__(self, module: nn.Linear, rank: int):
        super().__init__()
        self.module = module
        self.adapter_A = nn.Parameter(torch.empty(module.in_features, rank, device=module.weight.device))
        nn.init.kaiming_uniform_(self.adapter_A, a=5 ** 0.5)
        self.adapter_B = nn.Parameter(torch.zeros(rank, module.out_features, device=module.weight.device))

    def forward(self, input):
        # Apply self.module and LoRA adapter, return the sum (base module outputs + adapter outputs)
        #  <YOUR CODE HERE>
        # return ...


### Apply LoRA to the model

The code below applies LoRA adapters on top of Q/K/V linear layers in Llama attention. You may also choose to modify other layers:
* self_attn.o_proj - attention output projection
* mlp.up_proj, mlp.gate_proj, mlp.down_proj - transformer feedforward layers
* lm_head - output LM head

__Note:__ please scroll down for the homework task

In [ ]:
lora_rank = 8

for name, module in model.model.layers.named_modules():
    if 'LlamaDecoderLayer' in repr(type(module)):
        module.self_attn.q_proj = LoRALayer(module.self_attn.q_proj, rank=lora_rank)
        module.self_attn.k_proj = LoRALayer(module.self_attn.k_proj, rank=lora_rank)
        module.self_attn.v_proj = LoRALayer(module.self_attn.v_proj, rank=lora_rank)            

assert sum(isinstance(module, LoRALayer) for module in model.modules()) == 96  # for Llama-7B

In [ ]:
batch = tokenizer("This model wants to share its greatest secret:", return_tensors='pt', return_token_type_ids=False)
# test a single training step, make sure we get meaningful gradients
with torch.cuda.amp.autocast():
    out = model.forward(**batch)
    out.logits.norm().backward()

for i, module in enumerate(model.modules()):
    if isinstance(module, LoRALayer):
        assert module.adapter_B.grad is not None
        assert module.adapter_B.grad.norm().item() > 0

model.zero_grad(set_to_none=True)

### (example) How to train your model

The example below shows how to train the LoRA adapters on a dummy dataset. You will need to run a _similar_ training task later.

__Note:__ please scroll down for the homework task

In [ ]:
data = load_dataset("Abirate/english_quotes")
data = data.map(lambda samples: tokenizer(samples['quote']), batched=True)

In [ ]:
# checking if the model can learn. Change max_steps for proper training

trainer = transformers.Trainer(
    model=model, train_dataset=data['train'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4, gradient_accumulation_steps=1,
        warmup_steps=250, max_steps=5, learning_rate=2e-4, fp16=True,
        logging_steps=1, output_dir='outputs', report_to=None),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
# model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

### Now you train the model (3 points)

Your task is to fine-tune the model to _generate python code_. Please use the above examples for inspiration. More specifically,

* __dataset:__ use [codeparrot-clean](https://huggingface.co/datasets/codeparrot/codeparrot-clean) or any other data containing python code. Since you do not need much data for this excercise, it is enough to use just shorter validation subset of `codeparrots`
* __preprocessing:__ select python code based on file extentions (.py)  (may skip in case of codeparrot - it is 100% python)
* __short lines:__ please take the first 512 characters of each line
* __adapter type:__ please use LoRA as defined above __plus at least one of:__
   - extra adapter on lm_head
   - extra adapter on MLP components (mlp.*)
   - trainable input embeddings (requires tweaking memory usage)
* __training:__ you do not have to train to convergence. If all goes well, your model should `.generate` code after 500 steps. Please use batch size of at least 4 (4 x 1 x 512 tokens) using `gradient_accumulation_steps=4`.



In [ ]:
prompts =  ['', 'import', 'from', 'while', 'try', 'if', 'for', 'torch']  # add few more that are not 100% assiciated with Python

<A WHOLE LOT OF YOUR CODE>
# generate baseline samples with the selected prompts before finetuning
# please feel free to use transformers.Trainer (as above) or your custom training code
# after the training concludes, please show examples of text generated by your model. It is expected to look like Python code fragments
# print the generation examples nicely (suggestion: use pandas or HTML) for easier comparison
# note: your LoRA-enhanced model can run generation the same way as the non-trained model (above)

If you reach this: congratulations! you've completed everything in this practice session.

If you want to dig deeper, try to implement prompt-tuning (for bonus points!).
You can read more about prompt tuning variants in paper [1](https://arxiv.org/abs/2104.08691) or paper [2](https://arxiv.org/abs/2101.00190). Both versions can be implemented by passing trainable prompts as `model.forward(..., past_key_values=your_prompts)`. 



### Read more

* How post-training quantization works: https://arxiv.org/abs/2208.07339 
* An overview of running large models: https://huggingface.co/docs/accelerate/package_reference/big_modeling 
* A general library for different adapter types: https://adapterhub.ml/

### [extra info] How to optimize for inference

The code below converts training-optimized 8bit weights into inference-optimized layout. It should result in significantly faster inference in the same memory footprint. 
However, if you do this, you can no longer run training --
 there is no way to un-convert after the first optimized forward!

```python
model.config.use_cache = True
for module in model.modules():
    if isinstance(module, bnb.nn.Linear8bitLt):
        module.state.memory_efficient_backward = False
```

### [extra info] Running other models.

This notebook's code can run with other models of similar size. Most comparable are OPT-6.7b and Falcon-7B. They will require minor code tweaks:
1) indicate the model name in `AutoModelForCausalLM.from_pretrained()`
2) change tokenizer class to `AutoTokenizer`
3) change code to force gradient_required:
```
        class AddInputGrad(nn.Sequential):
            def forward(self, x): 
                return super().forward(x).requires_grad_(True)

        if 'facebook/opt' in model_name:
            model.model.decoder.project_in = lambda x: x.requires_grad_(True)
        elif 'falcon' in model_name:
            model.transformer.word_embeddings = AddInputGrad(model.transformer.word_embeddings)
```

4) change code to add Lora layers - need to refer to the transformer block components by their relevant names
